## Libraries

In [74]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus
import itertools
from datetime import datetime, timedelta
from tabulate import tabulate
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import pandas as pd

## Data

In [77]:
teams = ['Australia', 'England', 'Pakistan', 'India', 'Bangladesh', 'Sri Lanka', 'Afghanistan', 'New Zealand', 'South Africa', 'Netherlands']
venues = ['Ahmedabad', 'Bengaluru', 'Chennai', 'Delhi', 'Lucknow', 'Mumbai', 'Pune', 'Hyderabad', 'Jaipur']
days = [datetime.strptime("01-March-2025", "%d-%B-%Y") + timedelta(days=i) for i in range(45)]

venue_distances = {
    'Ahmedabad': {'Bengaluru': 1233, 'Chennai': 1369, 'Delhi': 776, 'Lucknow': 946, 'Mumbai': 439, 'Pune': 517, 'Hyderabad': 879, 'Jaipur': 525},
    'Bengaluru': {'Ahmedabad': 1233, 'Chennai': 291, 'Delhi': 1743, 'Lucknow': 1576, 'Mumbai': 844, 'Pune': 734, 'Hyderabad': 498, 'Jaipur': 1580},
    'Chennai': {'Ahmedabad': 1369, 'Bengaluru': 291, 'Delhi': 1761, 'Lucknow': 1526, 'Mumbai': 1032, 'Pune': 914, 'Hyderabad': 514, 'Jaipur': 1790},
    'Delhi': {'Ahmedabad': 776, 'Bengaluru': 1743, 'Chennai': 1761, 'Lucknow': 432, 'Mumbai': 1150, 'Pune': 1176, 'Hyderabad': 1262, 'Jaipur': 237},
    'Lucknow': {'Ahmedabad': 946, 'Bengaluru': 1576, 'Chennai': 1526, 'Delhi': 432, 'Mumbai': 1194, 'Pune': 1175, 'Hyderabad': 1078, 'Jaipur': 670},
    'Mumbai': {'Ahmedabad': 439, 'Bengaluru': 844, 'Chennai': 1032, 'Delhi': 1150, 'Lucknow': 1194, 'Pune': 121, 'Hyderabad': 623, 'Jaipur': 910},
    'Pune': {'Ahmedabad': 517, 'Bengaluru': 734, 'Chennai': 914, 'Delhi': 1176, 'Lucknow': 1175, 'Mumbai': 121, 'Hyderabad': 507, 'Jaipur': 1030},
    'Hyderabad': {'Ahmedabad': 879, 'Bengaluru': 498, 'Chennai': 514, 'Delhi': 1262, 'Lucknow': 1078, 'Mumbai': 623, 'Pune': 507, 'Jaipur': 1390},
    'Jaipur': {'Ahmedabad': 525, 'Bengaluru': 1580, 'Chennai': 1790, 'Delhi': 237, 'Lucknow': 670, 'Mumbai': 910, 'Pune': 1030, 'Hyderabad': 1390}
}

team_details = {
    "Australia": {
        "Captain": "Pat Cummins",
        "Coach": "Andrew McDonald",
        "Recent Performance": "Achieved a remarkable five-wicket victory over England in the Champions Trophy.",
        "Key Players": ["Josh Inglis", "Glenn Maxwell"],
        "World Ranking": 3
    },
    "England": {
        "Captain": "Jos Buttler",
        "Coach": "Matthew Mott",
        "Recent Performance": "Despite Ben Duckett's impressive 165 runs, England faced a five-wicket defeat against Australia in the Champions Trophy.",
        "Key Players": ["Ben Duckett", "Joe Root"],
        "World Ranking": 5
    },
    "Pakistan": {
        "Captain": "Babar Azam",
        "Coach": "Saqlain Mushtaq",
        "Recent Performance": "Reigning champions of the 2017 Champions Trophy, entering 2025 with a strong batting lineup.",
        "Key Players": ["Babar Azam", "Shaheen Afridi"],
        "World Ranking": 4
    },
    "India": {
        "Captain": "Rohit Sharma",
        "Coach": "Rahul Dravid",
        "Recent Performance": "India boasts a formidable batting lineup and aims to secure another ICC trophy since their 2013 Champions Trophy win.",
        "Key Players": ["Virat Kohli", "Jasprit Bumrah"],
        "World Ranking": 2
    },
    "Bangladesh": {
        "Captain": "Tamim Iqbal",
        "Coach": "Chandika Hathurusingha",
        "Recent Performance": "Facing challenges in adapting to aggressive ODI strategies, Bangladesh aims to make a significant impact.",
        "Key Players": ["Shakib Al Hasan", "Mushfiqur Rahim"],
        "World Ranking": 7
    },
    "Sri Lanka": {
        "Captain": "Dasun Shanaka",
        "Coach": "Chris Silverwood",
        "Recent Performance": "Sri Lanka is focusing on rebuilding and aims to showcase strong performances in the tournament.",
        "Key Players": ["Kusal Perera", "Wanindu Hasaranga"],
        "World Ranking": 8
    },
    "Afghanistan": {
        "Captain": "Hashmatullah Shahidi",
        "Coach": "Jonathan Trott",
        "Recent Performance": "Making their Champions Trophy debut, Afghanistan aims to continue their impressive performances from the 2023 ODI World Cup.",
        "Key Players": ["Rashid Khan", "Mohammad Nabi"],
        "World Ranking": 9
    },
    "New Zealand": {
        "Captain": "Mitchell Santner",
        "Coach": "Gary Stead",
        "Recent Performance": "Entering the tournament with high confidence from recent successes, including winning a tri-nation series in Pakistan.",
        "Key Players": ["Kane Williamson", "Trent Boult"],
        "World Ranking": 1
    },
    "South Africa": {
        "Captain": "Temba Bavuma",
        "Coach": "Rob Walter",
        "Recent Performance": "South Africa aims to capitalize on the inconsistency of other Group B teams to advance in the tournament.",
        "Key Players": ["Quinton de Kock", "Kagiso Rabada"],
        "World Ranking": 6
    },
    "Netherlands": {
        "Captain": "Scott Edwards",
        "Coach": "Ryan Campbell",
        "Recent Performance": "The Netherlands is focusing on gaining valuable experience and aims to cause upsets in the tournament.",
        "Key Players": ["Max O'Dowd", "Fred Klaassen"],
        "World Ranking": 14
    }
}

venue_details = {
    "Ahmedabad": {
        "Location": "Narendra Modi Stadium, Ahmedabad, India",
        "Capacity": 110000,
        "Pitch Type": "Batting-friendly",
        "Weather Forecast": "Hot with plenty of sun, 36°C",
        "Historical Matches": ["Hosted the 2022 IPL final and multiple international fixtures"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Bengaluru": {
        "Location": "M. Chinnaswamy Stadium, Bengaluru, India",
        "Capacity": 40000,
        "Pitch Type": "Balanced",
        "Weather Forecast": "Hot with hazy sun, 36°C",
        "Historical Matches": ["Hosted numerous international matches, including Test, ODI, and T20I fixtures"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Chennai": {
        "Location": "M. A. Chidambaram Stadium, Chennai, India",
        "Capacity": 50000,
        "Pitch Type": "Spin-friendly",
        "Weather Forecast": "Mostly sunny, 33°C",
        "Historical Matches": ["Hosted the second-ever tied Test match in 1986"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Delhi": {
        "Location": "Arun Jaitley Stadium, Delhi, India",
        "Capacity": 41820,
        "Pitch Type": "Slow and low, assists spinners",
        "Weather Forecast": "Plenty of sunshine, 26°C",
        "Historical Matches": ["Hosted multiple World Cup fixtures"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Lucknow": {
        "Location": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow, India",
        "Capacity": 50000,
        "Pitch Type": "Balanced",
        "Weather Forecast": "Hazy sunshine, 28°C",
        "Historical Matches": ["Hosted international matches and IPL fixtures"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Mumbai": {
        "Location": "Wankhede Stadium, Mumbai, India",
        "Capacity": 33000,
        "Pitch Type": "Offers bounce and pace, aiding fast bowlers",
        "Weather Forecast": "Hazy sun and hot, 35°C",
        "Historical Matches": ["Venue for the 2011 ICC Cricket World Cup Final"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Pune": {
        "Location": "Maharashtra Cricket Association Stadium, Pune, India",
        "Capacity": 37000,
        "Pitch Type": "Batting-friendly with consistent bounce",
        "Weather Forecast": "Hot with hazy sun, 36°C",
        "Historical Matches": ["Hosted international matches and IPL games since 2012"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Hyderabad": {
        "Location": "Rajiv Gandhi International Cricket Stadium, Hyderabad, India",
        "Capacity": 55000,
        "Pitch Type": "Batting-friendly with spin assistance",
        "Weather Forecast": "Hazy sunshine, 34°C",
        "Historical Matches": ["Hosted several international matches and IPL finals"],
        "Facilities": ["Parking", "Food Stalls"]
    },
    "Jaipur": {
        "Location": "Sawai Mansingh Stadium, Jaipur, India",
        "Capacity": 30000,
        "Pitch Type": "Balanced",
        "Weather Forecast": "Hazy sunshine, 27°C",
        "Historical Matches": ["Hosted international matches and is the home ground for Rajasthan Royals in the IPL"],
        "Facilities": ["Parking", "Food Stalls"]
    }
}

## Schedule Builder

In [80]:
# Model setup
model = LpProblem("Tournament_Scheduling", LpMinimize)

# Variables
x = LpVariable.dicts("Match", [(t1, t2, v, d) for t1, t2 in itertools.combinations(teams, 2) 
                              for v in venues for d in days], cat='Binary')

# Objective
model += lpSum(venue_distances[v1][v2] * x[t1, t2, v2, d] 
              for v1 in venues for v2 in venues for t1, t2 in itertools.combinations(teams, 2) 
              if v2 in venue_distances[v1] for d in days)

# Constraints
# 1. Each match occurs exactly once
for (t1, t2) in itertools.combinations(teams, 2):
    model += lpSum(x[t1, t2, v, d] for v in venues for d in days) == 1

# 2. Each venue hosts exactly 5 matches
for v in venues:
    model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) for d in days) == 5

# 3. Each team plays exactly 9 matches
for t in teams:
    model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) 
                  if t in (t1, t2) for v in venues for d in days) == 9

# 4. Minimum 4-day gap between a team's matches
# 4. Minimum 4-day gap between a team's matches
for t in teams:
    for d in days[:-3]:  # Ensure we're not going out of bounds
        model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) if t in (t1, t2) for v in venues) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=1)] for t1, t2 in itertools.combinations(teams, 2) if t in (t1, t2) for v in venues) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=2)] for t1, t2 in itertools.combinations(teams, 2) if t in (t1, t2) for v in venues) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=3)] for t1, t2 in itertools.combinations(teams, 2) if t in (t1, t2) for v in venues) <= 1

# 5. Corrected: Minimum 3-day gap between venue usage
# Ensure no matches within 2 days after a venue is used
for v in venues:
    for d in days[:-2]:  # Check each day with the next 2 days
        model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2)) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=1)] for t1, t2 in itertools.combinations(teams, 2)) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=2)] for t1, t2 in itertools.combinations(teams, 2)) <= 1

# 6. At most one match per day
for d in days:
    model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) for v in venues) <= 1

# 7. Each team plays at most once per venue
for t in teams:
    for v in venues:
        model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) 
                      if t in (t1, t2) for d in days) <= 1

# 8. No consecutive day matches for teams
for t in teams:
    for d in days[:-1]:
        model += lpSum(x[t1, t2, v, d] for t1, t2 in itertools.combinations(teams, 2) 
                      if t in (t1, t2) for v in venues) + \
                 lpSum(x[t1, t2, v, d+timedelta(days=1)] for t1, t2 in itertools.combinations(teams, 2) 
                      if t in (t1, t2) for v in venues) <= 1
# Solve
model.solve()

# Display results
scheduled_matches = []
for (t1, t2, v, d) in x:
    if x[t1, t2, v, d].varValue == 1:
        scheduled_matches.append((d, f"{t1} vs {t2}", v))

scheduled_matches.sort(key=lambda match: match[0])

headers = ["Date", "Match (Team1 VS Team2)", "Venue"]
table = [(match[0].strftime('%d-%B-%Y'), match[1], match[2]) for match in scheduled_matches]

print(tabulate(table, headers, tablefmt="grid"))
print("Status:", LpStatus[model.status])

+---------------+-----------------------------+-----------+
| Date          | Match (Team1 VS Team2)      | Venue     |
+===============+=============================+===========+
| 01-March-2025 | England vs South Africa     | Ahmedabad |
+---------------+-----------------------------+-----------+
| 02-March-2025 | India vs New Zealand        | Mumbai    |
+---------------+-----------------------------+-----------+
| 03-March-2025 | Sri Lanka vs Afghanistan    | Pune      |
+---------------+-----------------------------+-----------+
| 04-March-2025 | Pakistan vs Netherlands     | Jaipur    |
+---------------+-----------------------------+-----------+
| 05-March-2025 | Australia vs England        | Delhi     |
+---------------+-----------------------------+-----------+
| 06-March-2025 | Bangladesh vs South Africa  | Lucknow   |
+---------------+-----------------------------+-----------+
| 07-March-2025 | India vs Afghanistan        | Jaipur    |
+---------------+-----------------------

## Details Finder/Distance Calculator

In [91]:
def calculate_distance(path):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += venue_distances[path[i]][path[i + 1]]
    return total_distance

def display_results(matches):
    for row in table_tree.get_children():
        table_tree.delete(row)
    
    for match in matches:
        table_tree.insert("", "end", values=(match[0].strftime("%d-%b-%Y"), match[1], match[2]))
    
    status_var.set(f"Matches Found: {len(matches)}")

def clear_results():
    display_results(scheduled_matches)
    search_var.set("")

def search_matches(*args):
    query = search_var.get().lower()
    filtered_matches = [match for match in scheduled_matches if query in match[1].lower() or query in match[2].lower() or query in match[0].strftime("%d-%b-%Y").lower()]
    display_results(filtered_matches)

def search_by_team(event):
    try:
        selected_team = teams_listbox.get(teams_listbox.curselection())
        search_var.set(selected_team)
        search_matches()
    except tk.TclError:
        pass

def search_by_venue(event):
    try:
        selected_venue = venues_listbox.get(venues_listbox.curselection())
        search_var.set(selected_venue)
        search_matches()
    except tk.TclError:
        pass

# def export_to_csv():
#     matches = [(table_tree.item(row)["values"]) for row in table_tree.get_children()]
#     df = pd.DataFrame(matches, columns=["Date", "Match", "Venue"])
#     df.to_csv("scheduled_matches.csv", index=False)
#     messagebox.showinfo("Export Successful", "Scheduled matches have been exported to scheduled_matches.csv")

def show_match_details(event):
    selected_item = table_tree.selection()
    if selected_item:
        match = table_tree.item(selected_item)["values"]
        match_date, match_teams, match_venue = match

        team1, team2 = match_teams.split(" vs ")
        team1_details = team_details.get(team1, {})
        team2_details = team_details.get(team2, {})
        venue_info = venue_details.get(match_venue, {})

        details_message = (
            f"Date: {match_date}\n"
            f"Match: {match_teams}\n"
            f"Venue: {match_venue}\n\n"
            f"--- Team 1: {team1} ---\n"
            f"Captain: {team1_details.get('Captain', 'N/A')}\n"
            f"Coach: {team1_details.get('Coach', 'N/A')}\n"
            f"Recent Performance: {team1_details.get('Recent Performance', 'N/A')}\n"
            f"Key Players: {', '.join(team1_details.get('Key Players', []))}\n"
            f"World Ranking: {team1_details.get('World Ranking', 'N/A')}\n\n"
            f"--- Team 2: {team2} ---\n"
            f"Captain: {team2_details.get('Captain', 'N/A')}\n"
            f"Coach: {team2_details.get('Coach', 'N/A')}\n"
            f"Recent Performance: {team2_details.get('Recent Performance', 'N/A')}\n"
            f"Key Players: {', '.join(team2_details.get('Key Players', []))}\n"
            f"World Ranking: {team2_details.get('World Ranking', 'N/A')}\n\n"
            f"--- Venue: {match_venue} ---\n"
            f"Location: {venue_info.get('Location', 'N/A')}\n"
            f"Capacity: {venue_info.get('Capacity', 'N/A')}\n"
            f"Pitch Type: {venue_info.get('Pitch Type', 'N/A')}\n"
            f"Weather Forecast: {venue_info.get('Weather Forecast', 'N/A')}\n"
            f"Historical Matches: {', '.join(venue_info.get('Historical Matches', []))}\n"
            f"Facilities: {', '.join(venue_info.get('Facilities', []))}\n"
        )

        messagebox.showinfo("Match Details", details_message)
    else:
        messagebox.showwarning("No Selection", "Please select a match to view details.")

def calculate_and_display_distance():
    selected_team = search_var.get()
    if not selected_team:
        messagebox.showwarning("No Team Selected", "Please select a team to calculate the distance.")
        return
    
    path = []
    for row in table_tree.get_children():
        match = table_tree.item(row)["values"]
        match_teams = match[1]
        match_venue = match[2]
        if selected_team in match_teams:
            path.append(match_venue)
    
    if len(path) < 2:
        messagebox.showwarning("Insufficient Venues", "Not enough venues found for the selected team to calculate the distance.")
        return
    
    total_distance = calculate_distance(path)
    messagebox.showinfo("Total Distance", f"The total distance for the selected team's path is {total_distance} Kms.")

# GUI Setup
root = tk.Tk()
root.title("Match Schedule Finder")
root.state('zoomed')  # Full screen
root.configure(bg="#d0e7f9")

style = ttk.Style(root)
style.theme_use("clam")
style.configure("Treeview.Heading", font=("Helvetica", 12, "bold"), background="#005b96", foreground="white")
style.configure("Treeview", font=("Helvetica", 10), rowheight=25, background="#b3d9ff", foreground="black", fieldbackground="#b3d9ff")

# Title Frame
title_frame = tk.Frame(root, bg="#d0e7f9")
title_frame.pack(pady=10)
tk.Label(title_frame, text="Match Schedule Finder", font=("Helvetica", 20, "bold"), bg="#d0e7f9").pack()

# Search Frame
search_frame = tk.Frame(root, bg="#d0e7f9")
search_frame.pack(pady=10)
tk.Label(search_frame, text="Search:", font=("Helvetica", 12), bg="#d0e7f9").pack(side="left", padx=5)
search_var = tk.StringVar()
search_entry = tk.Entry(search_frame, textvariable=search_var, font=("Helvetica", 10), width=30)
search_entry.pack(side="left", padx=5)
search_entry.bind("<KeyRelease>", search_matches)

# Main Frame
main_frame = tk.Frame(root, bg="#d0e7f9")
main_frame.pack(pady=10, fill="both", expand=True)

# Table Display
columns = ("Date", "Match", "Venue")
table_frame = tk.Frame(main_frame)
table_frame.pack(side="left", padx=10, pady=10, fill="both", expand=True)

table_tree = ttk.Treeview(table_frame, columns=columns, show='headings')

for col in columns:
    table_tree.heading(col, text=col)
    table_tree.column(col, width=180, anchor="center")

table_tree.pack(side="left", fill="both", expand=True)
table_tree.bind("<Double-1>", show_match_details)

# Scrollbar for the table
scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table_tree.yview)
table_tree.configure(yscroll=scrollbar.set)
scrollbar.pack(side="right", fill="y")

# Options Frame
options_frame = tk.Frame(main_frame, bg="#d0e7f9")
options_frame.pack(side="right", padx=10, pady=10, fill="y")

# Teams Listbox
teams_label = tk.Label(options_frame, text="Teams:", font=("Helvetica", 12), bg="#d0e7f9")
teams_label.pack(pady=5)
teams_listbox = tk.Listbox(options_frame, font=("Helvetica", 10), height=10)
teams_listbox.pack(pady=5)
teams_listbox.bind("<<ListboxSelect>>", search_by_team)
for team in teams:
    teams_listbox.insert(tk.END, team)

# Venues Listbox
venues_label = tk.Label(options_frame, text="Venues:", font=("Helvetica", 12), bg="#d0e7f9")
venues_label.pack(pady=5)
venues_listbox = tk.Listbox(options_frame, font=("Helvetica", 10), height=10)
venues_listbox.pack(pady=5)
venues_listbox.bind("<<ListboxSelect>>", search_by_venue)
for venue in venues:
    venues_listbox.insert(tk.END, venue)

# Distance Calculation Button
distance_button = tk.Button(options_frame, text="Calculate Distance", command=calculate_and_display_distance, font=("Helvetica", 12), bg="#005b96", fg="white")
distance_button.pack(pady=10)

# Status Bar
status_var = tk.StringVar()
status_bar = tk.Label(root, textvariable=status_var, font=("Helvetica", 10), bg="#d0e7f9", anchor="w")
status_bar.pack(side="bottom", fill="x")

# Display the initial schedule
display_results(scheduled_matches)

# Clear Button
clear_button = tk.Button(root, text="Clear", command=clear_results, font=("Helvetica", 12), bg="#005b96", fg="white")
clear_button.pack(pady=10)

# Export Button
# export_button = tk.Button(root, text="Export to CSV", command=export_to_csv, font=("Helvetica", 12), bg="#005b96", fg="white")
# export_button.pack(pady=10)

root.mainloop()

## Points Table

In [94]:
import tkinter as tk
from tkinter import ttk
from datetime import datetime

teams = sorted(set(team for match in scheduled_matches for team in match[1].split(" vs ")))
points_table = {team: 0 for team in teams}
current_match_index = 0
semifinals = []
finals = []

def update_points(winner):
    global current_match_index
    match = scheduled_matches[current_match_index]
    team_a, team_b = match[1].split(" vs ")
    
    if winner in ["A", "a"]:
        points_table[team_a] += 2
    elif winner in ["B", "b"]:
        points_table[team_b] += 2
    elif winner in ["T", "t"]:
        points_table[team_a] += 1
        points_table[team_b] += 1
    
    current_match_index += 1
    if current_match_index < len(scheduled_matches):
        display_match()
    else:
        display_points_table()

def display_match():
    match = scheduled_matches[current_match_index]
    match_label.config(text=f"Match {current_match_index + 1}: {match[1]} at {match[2]} on {match[0].strftime('%d-%b-%Y')}")
    team_a, team_b = match[1].split(" vs ")
    table_tree.insert("", "end", values=(team_a, team_b))
    table_tree.yview_moveto(1)

def display_points_table():
    for row in table_tree.get_children():
        table_tree.delete(row)
    
    sorted_points = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
    for i, (team, points) in enumerate(sorted_points):
        if i < 4:
            table_tree.insert("", "end", values=(team, points, "SemiFinals"))
        else:
            table_tree.insert("", "end", values=(team, points))
    
    match_label.config(text="Final Points Table")
    root.bind("<Return>", lambda event: setup_semifinals(sorted_points[:4]))

def setup_semifinals(top_4):
    global semifinals
    semifinals = [
        (top_4[0][0], top_4[2][0], "SemiFinal-1"),
        (top_4[1][0], top_4[3][0], "SemiFinal-2")
    ]
    clear_table()
    display_semifinal_match(0)

def display_semifinal_match(index):
    if index < len(semifinals):
        match = semifinals[index]
        match_label.config(text=f"{match[2]}: {match[0]} vs {match[1]}")
        table_tree.insert("", "end", values=(match[0], match[1]))
        table_tree.yview_moveto(1)
        root.bind("<Return>", lambda event: submit_semifinal_winner(index))
    else:
        display_final_points_table()

def submit_semifinal_winner(index):
    winner = winner_var.get().upper()
    if winner in ["A", "B", "T"]:
        match = semifinals[index]
        team_a, team_b = match[0], match[1]
        if winner == "A":
            points_table[team_a] += 2
        elif winner == "B":
            points_table[team_b] += 2
        elif winner == "T":
            points_table[team_a] += 1
            points_table[team_b] += 1
        winner_var.set("")
        clear_table()
        display_semifinal_match(index + 1)
    else:
        match_label.config(text="Invalid input! Choose from A, B, or T.")

def display_final_points_table():
    for row in table_tree.get_children():
        table_tree.delete(row)
    
    sorted_points = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
    for i, (team, points) in enumerate(sorted_points[:4]):
        if i < 2:
            table_tree.insert("", "end", values=(team, points, "Finals"))
        else:
            table_tree.insert("", "end", values=(team, points))
    
    match_label.config(text="Final Points Table")
    root.bind("<Return>", lambda event: setup_final(sorted_points[:2]))

def setup_final(top_2):
    global finals
    finals = [(top_2[0][0], top_2[1][0], "FINALS")]
    clear_table()
    display_final_match()

def display_final_match():
    match = finals[0]
    match_label.config(text=f"{match[2]}: {match[0]} vs {match[1]}")
    table_tree.insert("", "end", values=(match[0], match[1]))
    table_tree.yview_moveto(1)
    root.bind("<Return>", lambda event: submit_final_winner())

def submit_final_winner():
    winner = winner_var.get().upper()
    if winner in ["A", "B", "T"]:
        match = finals[0]
        team_a, team_b = match[0], match[1]
        if winner == "A":
            points_table[team_a] += 2
        elif winner == "B":
            points_table[team_b] += 2
        elif winner == "T":
            points_table[team_a] += 1
            points_table[team_b] += 1
        winner_var.set("")
        display_champion()
    else:
        match_label.config(text="Invalid input! Choose from A, B, or T.")

def display_champion():
    for row in table_tree.get_children():
        table_tree.delete(row)
    
    sorted_points = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
    champion = sorted_points[0][0]
    match_label.config(text=f"Champion: {champion}")
    table_tree.insert("", "end", values=(champion, "Champion"))

def clear_table():
    for row in table_tree.get_children():
        table_tree.delete(row)

def submit_winner():
    winner = winner_var.get().upper()
    if winner in ["A", "B", "T"]:
        update_points(winner)
    else:
        match_label.config(text="Invalid input! Choose from A, B, or T.")
    winner_var.set("")

# GUI Setup
root = tk.Tk()
root.title("Match Schedule and Points Table")
root.geometry("800x600")
root.configure(bg="#d0e7f9")

style = ttk.Style(root)
style.theme_use("clam")
style.configure("Treeview.Heading", font=("Helvetica", 12, "bold"), background="#005b96", foreground="white")
style.configure("Treeview", font=("Helvetica", 10), rowheight=25, background="#b3d9ff", foreground="black", fieldbackground="#b3d9ff")

# Title Frame
title_frame = tk.Frame(root, bg="#d0e7f9")
title_frame.pack(pady=10)
tk.Label(title_frame, text="Match Schedule and Points Table", font=("Helvetica", 20, "bold"), bg="#d0e7f9").pack()

# Match Display Frame
match_frame = tk.Frame(root, bg="#d0e7f9")
match_frame.pack(pady=10)
match_label = tk.Label(match_frame, text="", font=("Helvetica", 14), bg="#d0e7f9")
match_label.pack()

# Winner Input Frame
winner_frame = tk.Frame(root, bg="#d0e7f9")
winner_frame.pack(pady=10)
tk.Label(winner_frame, text="Who won? (Choose from A, B, or T):", font=("Helvetica", 12), bg="#d0e7f9").pack(side="left", padx=5)
winner_var = tk.StringVar()
winner_entry = tk.Entry(winner_frame, textvariable=winner_var, font=("Helvetica", 10), width=5)
winner_entry.pack(side="left", padx=5)
winner_entry.bind("<Return>", lambda event: submit_winner())
submit_button = tk.Button(winner_frame, text="Submit", command=submit_winner, font=("Helvetica", 12), bg="#005b96", fg="white")
submit_button.pack(side="left", padx=5)

# Table Display
columns = ("A", "B", "Status")
table_frame = tk.Frame(root)
table_frame.pack(pady=10, fill="both", expand=True)

table_tree = ttk.Treeview(table_frame, columns=columns, show='headings')

for col in columns:
    table_tree.heading(col, text=col)
    table_tree.column(col, width=180, anchor="center")

table_tree.pack(side="left", fill="both", expand=True)

# Status Bar
status_var = tk.StringVar()
status_bar = tk.Label(root, textvariable=status_var, font=("Helvetica", 10), bg="#d0e7f9", anchor="w")
status_bar.pack(side="bottom", fill="x")

# Display the first match
display_match()

root.mainloop()